# Task C

In [3]:
!pip install transformers
import pandas as pd
import numpy as np
from torchtext.data.utils import get_tokenizer
import torch
import torchtext
import random
import nltk
from torch import nn 
import torch.nn.functional as F
torch.manual_seed(1)
np.random.seed(1)
from torch import utils
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from transformers import BertTokenizer
bert_tokenizer=BertTokenizer.from_pretrained('bert-base-cased')

In [4]:
train=pd.read_csv('train.csv')
train=train.copy()
train.head()

rating                                         reviewText  \
0       3  Plot Storyline: 5 StarsThis novel accomplished...   
1       3  I did not like how EL ended this one. I don't ...   
2       5  I love how old fashioned this family is - they...   
3       5  I loved this story - It's about two friends wh...   
4       1  In the Dark Lands, a virus killed all possibil...   

                                   summary  
0                              3 1/4 Stars  
1  It was going great, then just.... ended  
2                             LOVED ALL 4!  
3             friends make the best lovers  
4          Blatantly sexist and homophobic

In [5]:
train=pd.read_csv('train.csv')
train['text']=train['summary']+' '+train['reviewText'] #combine all the text features  
text=train['text']
train['rating'].unique() #The result of classification will start from zero rather than 1, therefore the ratings are re-coded here for convinence in the following steps.
train['response']=train['rating']-1
train.drop(['reviewText','summary','rating'],axis=1,inplace=True)#drop redundant columns

In [6]:
df_b=train.copy()
ind_train_b, ind_valid_b  = train_test_split(np.array(df_b.index), train_size=0.8, random_state=1)
train_b = df_b.loc[ind_train_b,:].copy()
valid_b = df_b.loc[ind_valid_b,:].copy()

In [7]:
from torch.utils.data import Dataset
# define the torch dataset，the code below is adapted from QBUS6850 tut6.
class MyDataset(Dataset):
    def __init__(self, myData):
        super().__init__()
        self.data = myData
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        if self.data.shape[1]==2:
            return (self.data.iloc[idx,0], self.data.iloc[idx,1]) # (text, response)
        else:
            return (self.data.iloc[idx,0])#(It is for only text)

In [8]:
train_torch = MyDataset(train_b)
val_torch = MyDataset(valid_b)

In [9]:
from torch.nn.utils.rnn import pad_sequence

def collate_bert(batch):
    
    text_list,response_list,mask_list = [], [], []
    
    # loop all sentence in the batch, combine text, response and mask
    for idx in range(len(batch)):
        
        label = batch[idx][1]
        text = batch[idx][0]
        
        response_list.append( label )
        tokens = bert_tokenizer(text,padding='max_length',max_length=512,truncation=True,return_tensors="pt")['input_ids'].squeeze(0)
        mask = bert_tokenizer(text,padding='max_length',max_length=512,truncation=True,return_tensors="pt")['attention_mask'].squeeze(0)
        text_list.append(tokens)
        mask_list.append(mask)   
    response_list = torch.tensor(response_list, dtype=torch.int64).squeeze(0)
    
    
    return text_list,mask_list,response_list

In [10]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
torch.manual_seed(0)

batchSize = 2 #in case the gpu of colab is out of memory, we have to use such a small batch size
train_loader1 = utils.data.DataLoader(train_torch, batch_size=batchSize, shuffle=True, collate_fn=collate_bert)
val_loader1 = utils.data.DataLoader(val_torch, batch_size=batchSize, shuffle=True, collate_fn=collate_bert)

In [13]:
from transformers import BertModel
import torch.nn as nn
class Bertclassifiy(nn.Module):
    def __init__(self, embed_dim, hidden_dim, output_dim):
        super().__init__()
        self.hidden_dim = hidden_dim       
        self.bert = BertModel.from_pretrained('bert-base-cased')#import bert as embedding layer before using LSTM 
        for param in self.bert.parameters():
            param.requires_grad = True
        self.lstm = nn.LSTM(embed_dim, hidden_dim,batch_first = True) 
        self.linear = nn.Linear(hidden_dim*2, output_dim)
        self.Dropout = nn.Dropout(p=0.5)
        
    def forward(self, input_id,mask):
        # embedding layer 
        encoder, pooled_output=self.bert(input_ids=input_id,attention_mask=mask,return_dict=False)
        lstm_out, (h_n, c_n) = self.lstm(encoder)
        hidden_last_F=h_n[-1]
        hidden_last_R=c_n[-1]
            # concat forward last and reverse last layers 
        hidden_out=torch.cat([hidden_last_F,hidden_last_R],dim=-1)
        hidden_out = self.Dropout(hidden_out)

        hidden_out = self.Dropout(hidden_out)
        # ast hidden state 
        out = self.linear(hidden_out)
        return out
       
        

embed_dim =  768 #embedding dim of bert is fixed to be 768
hidden_dim = 8 #many hidden layers lead to slow speed of training
output_dim = 5
model = Bertclassifiy(embed_dim, hidden_dim, output_dim).to(device)


Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.000001)

In [1]:
# code below is adapted from the QBUS6850 tut6
def evaluate_bert(dataloader):
    
    y_pred = torch.tensor([]).to(device) # prediction
    y_true = torch.tensor([]).to(device) # true value
    
    model.eval()
    with torch.no_grad():
        for text, mask,label in dataloader:
            text=torch.stack(text, dim=0).to(device)
            mask=torch.stack(mask, dim=0).to(device)
            y_pred_batch = model(text,mask).to(device)
            
            y_pred = torch.cat((y_pred, y_pred_batch.squeeze()))
            y_true = torch.cat((y_true, label.squeeze().to(device)))
    return y_pred, y_true

In [18]:
# the code must be run in the colab because gpu resource of our PCs is limited. There might be an error if cuda is out of memory. 
epochs =15 
for epoch in range(epochs): 
    for text,mask,y_train in train_loader1:
        model.train()
        optimizer.zero_grad()
        text=torch.stack(text, dim=0).to(device)
        mask=torch.stack(mask, dim=0).to(device)
        y_train=y_train.to(device)
        # calulate prediction of y 
        y_pred_train = model(text,mask).to(device)
        #print(y_pred_train)
        #print(y_train)
        loss=loss_fn(y_pred_train.view(-1, y_pred_train.shape[-1]), y_train.view(-1))
      

        # backprop and take a step
        loss.backward()
        optimizer.step()
     
    # evaluation
    y_pred_val, y_val = evaluate_bert(val_loader1)
    loss_val = loss_fn(y_pred_val.squeeze(), y_val.long())

    pred_label = torch.argmax( y_pred_val, -1).long() # use argmax to find which label has the largest probability
    acc = (pred_label == y_val).float().sum()/y_val.shape[0]
    f1 = f1_score(pred_label.cpu(), y_val.cpu().numpy(), average = 'macro') #print f1 score 
    print('Epoch {}: {:.4f} (train loss), {:.4f} (val loss), {:.4f} (val accuracy),{:.4f}(f1)'.format(epoch, loss, loss_val, acc,f1))

Epoch 0: 1.0440 (train), 1.1524 (val), 0.5483 (val acc),0.5161(f1)
Epoch 1: 1.0954 (train), 1.1807 (val), 0.5517 (val acc),0.5237(f1)
Epoch 2: 1.4883 (train), 1.1478 (val), 0.5472 (val acc),0.5271(f1)
Epoch 3: 1.7641 (train), 1.1144 (val), 0.5628 (val acc),0.5369(f1)
Epoch 4: 0.6032 (train), 1.1642 (val), 0.5422 (val acc),0.5158(f1)
Epoch 5: 1.2910 (train), 1.1826 (val), 0.5472 (val acc),0.5212(f1)
Epoch 6: 0.9311 (train), 1.1090 (val), 0.5689 (val acc),0.5485(f1)
Epoch 7: 1.5342 (train), 1.1135 (val), 0.5700 (val acc),0.5497(f1)
Epoch 8: 0.8971 (train), 1.1284 (val), 0.5567 (val acc),0.5323(f1)
Epoch 9: 1.0660 (train), 1.1668 (val), 0.5689 (val acc),0.5546(f1)
Epoch 10: 1.0411 (train), 1.3543 (val), 0.5711 (val acc),0.5451(f1)
Epoch 11: 1.3053 (train), 1.2856 (val), 0.5783 (val acc),0.5626(f1)
Epoch 12: 1.2512 (train), 1.4954 (val), 0.5772 (val acc),0.5617(f1)
Epoch 13: 1.6651 (train), 1.1930 (val), 0.5672 (val acc),0.5566(f1)
Epoch 14: 1.9112 (train), 1.2124 (val), 0.5728 (val acc),0